![Image of Yaktocat](https://raw.githubusercontent.com/pola-rs/polars-static/master/logos/polars-logo-dark.svg)

### What is Polars?
- Pandas alternative
- written in Rust
- uses Apache Arrow
- Polars uses parallelization, but not across machines

### What does Polars do differently than Pandas?
- Utilizes all available cores on your machine
- Optimizes queries to reduce unneeded work/memory allocations
- Handles datasets much larger than your available RAM

### Goals of this demonstration
- Show syntax, but you won't be fluent afterwards
- Show that polars is crazy fast
- Show that transitioning from pandas is easy

In [11]:
import sys
import polars as pl
import pandas as pd
import numpy as np

### Read Data in

In [12]:
%%time
pandas = (
    pd
    .read_csv("Data/Processed/combined_data_1.csv")
    .assign(date=lambda x: pd.to_datetime(x.date))
    .assign(year=lambda x: x.date.dt.year)
)

CPU times: user 12.8 s, sys: 8.49 s, total: 21.3 s
Wall time: 21.3 s


In [13]:
pandas.head()

,customer_id,rating,date,movie_id,year
0,1488844,3.0,2005-09-06,1,2005
1,822109,5.0,2005-05-13,1,2005
2,885013,4.0,2005-10-19,1,2005
3,30878,4.0,2005-12-26,1,2005
4,823519,3.0,2004-05-03,1,2004


In [7]:
%%time
polars = (
    pl
    .read_csv("Data/Processed/combined_data_1.csv")
    .with_columns(pl.col("date").str.strptime(pl.Date, fmt="%Y-%m-%d"))
    .with_columns(pl.col("date").dt.year().alias("year"))    
)

CPU times: user 9.48 s, sys: 1.47 s, total: 10.9 s
Wall time: 1.78 s


<timed exec>:2: DeprecationWarning: `DataFrame.with_column` has been renamed; this redirect is temporary, please use `.with_columns` instead


In [4]:
%%time
_ = (
    pl
    .read_csv("Data/Processed/combined_data_1.csv")
    .to_pandas()
    .assign(date=lambda x: pd.to_datetime(x.date))
    .assign(year=lambda x: x.date.dt.year) 
)

CPU times: user 20.7 s, sys: 2.69 s, total: 23.4 s
Wall time: 9.84 s


In [5]:
polars.shape

(24053764, 5)

In [6]:
print(polars.head())

shape: (5, 5)
┌─────────────┬────────┬────────────┬──────────┬──────┐
│ customer_id ┆ rating ┆ date       ┆ movie_id ┆ year │
│ ---         ┆ ---    ┆ ---        ┆ ---      ┆ ---  │
│ i64         ┆ f64    ┆ date       ┆ i64      ┆ i32  │
╞═════════════╪════════╪════════════╪══════════╪══════╡
│ 1488844     ┆ 3.0    ┆ 2005-09-06 ┆ 1        ┆ 2005 │
│ 822109      ┆ 5.0    ┆ 2005-05-13 ┆ 1        ┆ 2005 │
│ 885013      ┆ 4.0    ┆ 2005-10-19 ┆ 1        ┆ 2005 │
│ 30878       ┆ 4.0    ┆ 2005-12-26 ┆ 1        ┆ 2005 │
│ 823519      ┆ 3.0    ┆ 2004-05-03 ┆ 1        ┆ 2004 │
└─────────────┴────────┴────────────┴──────────┴──────┘


In [7]:
sys.getsizeof(pandas)

962150704

In [8]:
sys.getsizeof(polars)

48

### Selecting data

#### Selecting rows

In [9]:
%%time
pandas[pandas["year"]==2002].head()

CPU times: user 991 ms, sys: 472 ms, total: 1.46 s
Wall time: 1.46 s


,customer_id,rating,date,movie_id,year
5009,2056324,2.0,2002-11-10,7,2002
5022,1876520,1.0,2002-06-15,7,2002
5028,1596032,1.0,2002-02-14,7,2002
5034,507140,2.0,2002-06-06,7,2002
5035,1901461,1.0,2002-11-09,7,2002


In [10]:
%%time
polars.filter(pl.col("year")==2002).head()

CPU times: user 429 ms, sys: 8.93 ms, total: 438 ms
Wall time: 232 ms


customer_id,rating,date,movie_id,year
i64,f64,date,i64,i32
2056324,2.0,2002-11-10,7,2002
1876520,1.0,2002-06-15,7,2002
1596032,1.0,2002-02-14,7,2002
507140,2.0,2002-06-06,7,2002
1901461,1.0,2002-11-09,7,2002


#### Selecting columns

In [11]:
pandas[["movie_id", "rating"]].head()

,movie_id,rating
0,1,3.0
1,1,5.0
2,1,4.0
3,1,4.0
4,1,3.0


In [12]:
polars.select(["movie_id", "rating"]).head()

movie_id,rating
i64,f64
1,3.0
1,5.0
1,4.0
1,4.0
1,3.0


#### Polars can also do regex

In [13]:
%%time
movie_titles = pd.read_csv("Data/movie_titles.csv").dropna()
movie_titles[movie_titles.movie_title.str.contains("Lord.*Rings")]

CPU times: user 49.6 ms, sys: 0 ns, total: 49.6 ms
Wall time: 45.8 ms


,movie_id,year,movie_title
12,13,2003.0,Lord of the Rings: The Return of the King: Ext...
1600,1601,2004.0,J.R.R. Tolkien and the Birth of The Lord of th...
1756,1757,1978.0,The Lord of the Rings
2451,2452,2001.0,Lord of the Rings: The Fellowship of the Ring
3490,3491,2001.0,National Geographic: Beyond the Movie: The Lor...
7056,7057,2002.0,Lord of the Rings: The Two Towers: Extended Ed...
7229,7230,2001.0,The Lord of the Rings: The Fellowship of the R...
8090,8091,2002.0,Lord of the Rings: The Two Towers: Bonus Material
10312,10313,2001.0,Lord of the Rings: The Fellowship of the Ring:...
10351,10352,2003.0,Lord of the Rings: The Return of the King: Bon...


In [14]:
%%time
(
    pl
    .read_csv("Data/movie_titles.csv", has_header=True)
    .filter(pl.col("movie_title").str.contains("Lord.*Rings"))

)

CPU times: user 24.6 ms, sys: 7.21 ms, total: 31.8 ms
Wall time: 10.5 ms


movie_id,year,movie_title
i64,i64,str
13,2003,"""Lord of the Ri..."
1601,2004,"""J.R.R. Tolkien..."
1757,1978,"""The Lord of th..."
2452,2001,"""Lord of the Ri..."
3491,2001,"""National Geogr..."
7057,2002,"""Lord of the Ri..."
7230,2001,"""The Lord of th..."
8091,2002,"""Lord of the Ri..."
10313,2001,"""Lord of the Ri..."


### Polars is lazy

Two modes:
- lazy
- eager

pandas is eager, polas per default too,

Lazy mode can be either
- implicit (lazy() + collect()) (see an example further down under 'Joining Data & Lazy Mode')
- explicit (e.g. pl.scan_csv())

In [23]:
%%time
(
    pd
    .read_csv("Data/Processed/combined_data_1.csv")
    .assign(date=lambda x: pd.to_datetime(x.date))
    .assign(year=lambda x: x.date.dt.year)
    .groupby("year")
    .agg({"rating": "mean"})
    .sort_values("year")
)

CPU times: user 14.5 s, sys: 2.33 s, total: 16.8 s
Wall time: 16.8 s


,rating
year,
1999,3.107981
2000,3.311930
2001,3.337481
2002,3.347615
2003,3.398940
2004,3.592790
2005,3.673497


In [21]:
%%time
(
    pl
    .read_csv("Data/Processed/combined_data_1.csv")
    .with_columns(pl.col("date").str.strptime(pl.Date, fmt="%Y-%m-%d"))
    .with_columns(pl.col("date").dt.year().alias("year"))
    .groupby("year")
    .agg(pl.mean("rating"))
    .sort("year")
)

CPU times: user 13.7 s, sys: 3.46 s, total: 17.2 s
Wall time: 3.3 s


year,rating
i32,f64
1999,3.107981
2000,3.31193
2001,3.337481
2002,3.347615
2003,3.39894
2004,3.59279
2005,3.673497


In [17]:
%%time
(
    pl
    .scan_csv("Data/Processed/combined_data_1.csv")
    .with_columns(pl.col("date").str.strptime(pl.Date, fmt="%Y-%m-%d"))
    .with_columns(pl.col("date").dt.year().alias("year"))
    .filter(pl.col("year")>=2001)
    .groupby("year")
    .agg(pl.mean("rating"))
    .sort("year")
    .collect()
)

CPU times: user 10.8 s, sys: 611 ms, total: 11.4 s
Wall time: 1.25 s


year,rating
i32,f64
2001,3.337481
2002,3.347615
2003,3.39894
2004,3.59279
2005,3.673497


In [16]:
%%time
df = (
    pl
    .scan_csv("Data/Processed/combined_data_1.csv")
    .with_columns(pl.col("date").str.strptime(pl.Date, fmt="%Y-%m-%d"))
    .with_columns(pl.col("date").dt.year().alias("year"))
    .filter(pl.col("year")>=2001)
    .groupby("year")
    .agg(pl.mean("rating"))
    .sort("year")
#    .collect()
)
df

CPU times: user 2.64 ms, sys: 288 µs, total: 2.93 ms
Wall time: 2.71 ms


In [19]:
type(df)

polars.internals.lazyframe.frame.LazyFrame

### Polars uses multi-threading per default

In [24]:
%%time
pandas["month"] = pandas["date"].dt.month
pandas["day"] = pandas["date"].dt.day
pandas.head()

CPU times: user 5.51 s, sys: 153 ms, total: 5.67 s
Wall time: 6.57 s


,customer_id,rating,date,movie_id,year,month,day
0,1488844,3.0,2005-09-06,1,2005,9,6
1,822109,5.0,2005-05-13,1,2005,5,13
2,885013,4.0,2005-10-19,1,2005,10,19
3,30878,4.0,2005-12-26,1,2005,12,26
4,823519,3.0,2004-05-03,1,2004,5,3


In [25]:
%%time
polars = polars.with_columns([
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.day().alias("day"), 
    ])
polars.head()

CPU times: user 838 ms, sys: 83.8 ms, total: 922 ms
Wall time: 521 ms


customer_id,rating,date,movie_id,year,month,day
i64,f64,date,i64,i32,u32,u32
1488844,3.0,2005-09-06,1,2005,9,6
822109,5.0,2005-05-13,1,2005,5,13
885013,4.0,2005-10-19,1,2005,10,19
30878,4.0,2005-12-26,1,2005,12,26
823519,3.0,2004-05-03,1,2004,5,3


### Window functions

In [ ]:
# not executable. kernel keeps dying.

# %%time
# _ = (
#     pandas
#     .groupby("year")
#     .agg({"rating": "mean"})
#     .rename(columns={"rating": "avg_rating_per_year"})
#     .reset_index()
#     .merge(pandas, on="year")
#     .head()
# )

In [26]:
%%time
(
    polars
    .select([
        "movie_id",
        "customer_id",
        "rating",
        "year",
        pl.col("rating").mean().over(["year"]).alias("avg_rating_per_year"),
    ])
    .filter(pl.col("rating")>=pl.col("avg_rating_per_year"))
    .head()
)

CPU times: user 3.97 s, sys: 546 ms, total: 4.51 s
Wall time: 1.18 s


movie_id,customer_id,rating,year,avg_rating_per_year
i64,i64,f64,i32,f64
1,822109,5.0,2005,3.673497
1,885013,4.0,2005,3.673497
1,30878,4.0,2005,3.673497
1,124105,4.0,2004,3.59279
1,1842128,4.0,2004,3.59279


### Joining Data & Lazy Mode

In [2]:
movie_titles = pl.read_csv("Data/movie_titles.csv", has_header=True, ignore_errors=True)

### First eager

In [8]:
%%time
df = (
    pl
    .read_csv("Data/Raw/movie_titles.csv", has_header=False, ignore_errors=True, new_columns=["movie_id", "year", "movie_title"])
    .select(["movie_id", "movie_title", "year"])
    .join(
        (
            pl
            .read_csv("Data/Processed/combined_data_1.csv")
            .with_columns(pl.col("date").str.strptime(pl.Date, fmt="%Y-%m-%d"))
            .with_columns([
               # pl.col("date").dt.year().alias("year"),
                pl.col("date").dt.month().alias("month"),
            ])    
        )
        , on=["movie_id"])
    .sort(["rating", "year"])
#    .filter((pl.col("year")>=2000) & (pl.col("year")<=2004))
    .select([
        "movie_title",
        "year",
        "rating",
        pl.col("year").count().over(["movie_title"]).alias("rating_count"),
        
    ])
     .sort("rating_count")
    .filter(pl.col("rating_count")>=1000)
    .groupby(["movie_title", "year"])
    .agg([pl.col("rating").mean().alias("avg_rating")])
    .sort(pl.col("year") + pl.col("avg_rating"))
    .select([
        "movie_title",
        "year",
        "avg_rating",
        pl.col("avg_rating").rank(descending=True, method="min").over(["year"]).alias("movie_rating_rank_per_year"),
        
    ])
    .sort("movie_rating_rank_per_year")
   .filter(pl.col("movie_rating_rank_per_year")==1)
)
df

CPU times: user 3min 35s, sys: 36.1 s, total: 4min 11s
Wall time: 35.6 s


movie_title,year,avg_rating,movie_rating_rank_per_year
str,i64,f64,u32
"""20""",1916,3.572047,1
"""The Mark of Zo...",1920,3.395069,1
"""Battleship Pot...",1925,3.812357,1
"""The Little Ras...",1927,3.760887,1
"""Nosferatu: Ori...",1929,3.68638,1
"""Animal Cracker...",1930,4.016122,1
"""City Lights""",1931,4.071333,1
"""A Farewell to ...",1932,3.196172,1
"""Imitation of L...",1934,4.006108,1


### Now it's lazy

In [9]:
%%time
df = (
    pl
    .read_csv("Data/Raw/movie_titles.csv", has_header=False, ignore_errors=True, new_columns=["movie_id", "year", "movie_title"])
    .lazy()
    .select(["movie_id", "movie_title"])
    .join(
        (
            pl
            .read_csv("Data/Processed/combined_data_4.csv")
            .with_columns(pl.col("date").str.strptime(pl.Date, fmt="%Y-%m-%d"))
            .with_columns([
                pl.col("date").dt.year().alias("year"),
                pl.col("date").dt.month().alias("month"),
            ])    
            .lazy()
        )
        , on=["movie_id"])
    .sort(["rating", "year"])
    .filter((pl.col("year")>=2000) & (pl.col("year")<=2004))
    .select([
        "movie_title",
        "year",
        "rating",
        pl.col("year").count().over(["movie_title"]).alias("rating_count"),
        
    ])
     .sort("rating_count")
    .filter(pl.col("rating_count")>=1000)
    .groupby(["movie_title", "year"])
    .agg([pl.col("rating").mean().alias("avg_rating")])
    .sort(pl.col("year") + pl.col("avg_rating"))
    .select([
        "movie_title",
        "year",
        "avg_rating",
        pl.col("avg_rating").rank(descending=True, method="min").over(["year"]).alias("movie_rating_rank_per_year"),
        
    ])
    .sort("movie_rating_rank_per_year")
   .filter(pl.col("movie_rating_rank_per_year")==1)
   .collect()
)
df

CPU times: user 2min 18s, sys: 39.4 s, total: 2min 57s
Wall time: 26.5 s


movie_title,year,avg_rating,movie_rating_rank_per_year
str,i32,f64,u32
"""Great Expectat...",2000,5.0,1
"""Artie Lange: I...",2000,5.0,1
"""Outrageous For...",2001,5.0,1
"""Shakespeare Wa...",2001,5.0,1
"""Coolie No.1""",2001,5.0,1
"""Mr. & Mrs. Bri...",2001,5.0,1
"""Food of Love""",2002,5.0,1
"""Scratch""",2002,5.0,1
"""Count Basie at...",2002,5.0,1


### Weird things

#### Boolean columns can be generated just by passing a condition

In [16]:
df = (
    polars
    .with_columns([(pl.col("year")>2000).alias("gt_2000")])
)
df.head()

,movie_id,customer_id,rating,date,year,gt_2000
i64,i64,i64,f64,date,i32,bool
0,0,1488844,3.0,2005-09-06,2005,true
1,1,822109,5.0,2005-05-13,2005,true
2,2,885013,4.0,2005-10-19,2005,true
3,3,30878,4.0,2005-12-26,2005,true
4,4,823519,3.0,2004-05-03,2004,true


#### Boolean columns can be filtered just by passing a condition

In [18]:
df.filter(pl.col("gt_2000")).head()

,movie_id,customer_id,rating,date,year,gt_2000
i64,i64,i64,f64,date,i32,bool
0,0,1488844,3.0,2005-09-06,2005,true
1,1,822109,5.0,2005-05-13,2005,true
2,2,885013,4.0,2005-10-19,2005,true
3,3,30878,4.0,2005-12-26,2005,true
4,4,823519,3.0,2004-05-03,2004,true
